## Setup

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [1]:
# Acquring GPU
# !nvidia-smi

import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /Users/devasenan/Documents/projects/mfs/final_version/layout_estimator


In [2]:
LSUN_PATH = HOME+"/lsun-room/"

%cd {LSUN_PATH}
!pip install -r requirements.txt --pre

/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator/lsun-room
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Using cached kornia-0.4.1-py2.py3-none-any.whl (225 kB)
  Using cached opencv-python-4.5.1.48.tar.gz (88.3 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [1340 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_machine != "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.6" and sys_platform == "linux" and platform_machine == "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_machine != "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version

In [4]:
!pip install loguru
!pip install --upgrade grpcio
!pip install tensorboardx==2.1
# %load_ext tensorboard
!pip install pytorch-lightning
!pip install kornia

In [5]:
%cd {LSUN_PATH}
# !git clone https://github.com/leVirve/OneGAN.git
%cd OneGAN/
!python setup.py install

# %cd {LSUN_PATH}

/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator/lsun-room
Cloning into 'OneGAN'...
remote: Enumerating objects: 748, done.
remote: Total 748 (delta 0), reused 0 (delta 0), pack-reused 748
Receiving objects: 100% (748/748), 421.04 KiB | 1.59 MiB/s, done.
Resolving deltas: 100% (497/497), done.
/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator/lsun-room/OneGAN
running install
/Users/devasenan/opt/anaconda3/envs/torch/lib/python3.11/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/Users/devasenan/opt/anaconda3/envs/torch/lib/python3.11/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating onegan.egg-info
writing onegan.egg-info/PKG-INFO
writi

In [6]:
import onegan
%cd ../
%pwd

/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator/lsun-room


'/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator/lsun-room'

In [9]:
%cd {HOME}
image_filepath = f'../src_img/'
image_filepath

/Users/devasenan/Documents/projects/mfs/final_version/layout_estimator


'../src_img/'

## Process the input images from folder

In [ ]:
import numpy as np
import pathlib
from loguru import logger
import cv2
import matplotlib.pyplot as plt
import time

from demo import Predictor
from datasets import sequence

output_folder = f'../layout_output/'
weight='model_retrained.ckpt'
image_size = 320
path = image_filepath
cat_visual = False

output_folder = pathlib.Path(output_folder)
output_folder.mkdir(exist_ok=True, parents=True)

predictor = Predictor(weight_path=weight)
images = sequence.ImageFolder(image_size, path)
all_results = []

for image, shape, path in images:
    st = time.time()
    results = predictor.feed(image, alpha=1.)
    label = cv2.resize(results[0], shape)
    et = time.time()
    all_results.append(results)

    image = cv2.resize((image / 2 + .5).permute(1, 2, 0).numpy(), shape)
    if cat_visual:
        output = np.concatenate([image, label], axis=1)
    else:
        output = label
    output_path = output_folder / path.name
    cv2.imwrite(str(output_path), (output[..., ::-1] * 255).astype(np.uint8))

    output_matrix_path = str(output_folder)+'/'+path.name.split('.')[0]+'.txt'
    np.savetxt(output_matrix_path, results[2][0].cpu().data.numpy())

    logger.info('Write to {}, Exec time{:.3f}s'.format(output_path, et-st))

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.1.7 to v2.0.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../drive/MyDrive/Colab Notebooks/ML/projects/MFS/spatial layout mapping/model_retrained.ckpt`
2023-07-03 11:46:25.095 | INFO     | __main__:<cell line: 25>:43 - Write to /content/data/outputs/img9.jpeg, Exec time0.042s
2023-07-03 11:46:25.310 | INFO     | __main__:<cell line: 25>:43 - Write to /content/data/outputs/img1.jpeg, Exec time0.041s
2023-07-03 11:46:25.523 | INFO     | __main__:<cell line: 25>:43 - Write to /content/data/outputs/img2.jpeg, Exec time0.043s
2023-07-03 11:46:25.730 | INFO     | __main__:<cell line: 25>:43 - Write to /content/data/outputs/img6.jpeg, Exec time0.042s
2023-07-03 11:46:25.898 | INFO     | __main__:<cell line: 25>:43 - Write to /content/data/outputs/3.jpg, Exec time0.040s
2023-07-03 11:46:26.103 | INFO     | __m

In [ ]:
# b, _, out, lab = all_results[-1]